In [1]:
# Import libraries
import numpy as np
import MDAnalysis as mda
import nmrformd as nmrmd

recalculate=False

# For figure plotting
import sys
from matplotlib import pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
msize = 15
fontsize = 30
font = {'family': 'sans', 'color':  'black', 'weight': 'normal', 'size': fontsize}
myblue_light = [42/ 255, 99/255, 255/ 255] # [0/ 255, 150/255, 177/ 255]
myblue_dark = [43/ 255, 140/255, 238/ 255] # [0/ 255, 150/255, 177/ 255]
myred= [244/ 255, 143/255, 97/ 255] # [255/ 255, 77/255, 77/ 255]
mygreen = [122/255, 190/255, 165/255]
lightgray = [0.1, 0.1, 0.1]
darkgray = [0.9, 0.9, 0.9]
white = [0, 0, 0]
black = [1, 1, 1]
sys.path.append('../../shared/')
from tools import figure_improvement, save_figure

# Path to data
datapath = "../../raw-data/polymer-in-water/N17/"

if recalculate:
    # Create a MDAnalysis universe
    u = mda.Universe(datapath+"topology.data", datapath+"prod.xtc")
    #u.transfer_to_memory(stop=50) # to analyse only a subpart of the trajectory

    # Print a few information
    timestep = np.int32(u.trajectory.dt)
    print(f"The timestep is {timestep} ps")
    total_time = np.int32(u.trajectory.totaltime)
    print(f"The total simulation time is {total_time} ps")

    # cut-off for frequency and time
    f_1 = 1
    t_1 = 2
    if u.trajectory.n_frames == 10001:
        f_1 = 1600
        t_1 = 8000

    # Run NMRforMD
    group_H = u.select_atoms("type 3 5 7")
    group_H_water = u.select_atoms("type 7")
    group_H_polymer = u.select_atoms("type 3 5")
    n_water_molecules = group_H_water.n_residues
    print(f"The number of water molecules is {n_water_molecules}")
    n_polymer_molecules = group_H_polymer.n_residues
    print(f"The number of PEG molecules is {n_polymer_molecules}")
    
    PEG_nmr = nmrmd.NMR(u, group_H_polymer, neighbor_group=group_H, number_i=50)
    H2O_nmr = nmrmd.NMR(u, group_H_water, neighbor_group=group_H, number_i=50)

    T1_PEG = np.round(PEG_nmr.T1,2)
    print(f"NMR relaxation time T1 - PEG = {T1_PEG} s")
    T1_H2O = np.round(H2O_nmr.T1,2)
    print(f"NMR relaxation time T1 - H2O = {T1_H2O} s")

    f, R1_PEG = PEG_nmr.f[:-f_1], PEG_nmr.R1[:-f_1]
    np.savetxt("../../treated-data/tutorials/polymer-in-water/R1-PEG.dat", np.vstack([f, R1_PEG]).T)
    f, R1_H2O = H2O_nmr.f[:-f_1], H2O_nmr.R1[:-f_1]
    np.savetxt("../../treated-data/tutorials/polymer-in-water/R1-H2O.dat", np.vstack([f, R1_H2O]).T)
    t, gij_PEG = PEG_nmr.t[:-t_1], PEG_nmr.gij[0][:-t_1]
    np.savetxt("../../treated-data/tutorials/polymer-in-water/gij-PEG.dat", np.vstack([t, gij_PEG]).T)
    t, gij_H2O = H2O_nmr.t[:-t_1], H2O_nmr.gij[0][:-t_1]
    np.savetxt("../../treated-data/tutorials/polymer-in-water/gij-H2O.dat", np.vstack([t, gij_H2O]).T)
else:
    f, R1_PEG = np.loadtxt("../../treated-data/tutorials/polymer-in-water/R1-PEG.dat").T
    f, R1_H2O = np.loadtxt("../../treated-data/tutorials/polymer-in-water/R1-H2O.dat").T
    t, gij_H2O = np.loadtxt("../../treated-data/tutorials/polymer-in-water/gij-H2O.dat").T
    t, gij_H2O = np.loadtxt("../../treated-data/tutorials/polymer-in-water/gij-H2O.dat").T

The timestep is 1 ps
The total simulation time is 10000 ps
The number of water molecules is 85
The number of PEG molecules is 17


KeyboardInterrupt: 

In [ ]:
# R1 bulk
for mode, mygray, edge, myblue in zip(['light', 'dark'], [lightgray, darkgray], [black, white], [myblue_light, myblue_dark]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(f, R1_PEG, 's', markersize=msize, markeredgecolor=edge, markeredgewidth=0.2, color=myred, label=r'PEG')
    ax1.loglog(f, R1_H2O, 'o', markersize=msize, markeredgecolor=edge, markeredgewidth=0.2, color=myblue, label=r'H2O')
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$f$ (MHz)", ylabel=r'$R_1$ (s$^{-1}$)',
                       xlim=(5e1, 2e5), ylim=(0.01, 10))
    save_figure(plt, fig, mode, name='tutorials/polymer-in-water/R1', save=True)

In [ ]:
# Correlation function
for mode, mygray, edge, myblue in zip(['light', 'dark'], [lightgray, darkgray], [black, white], [myblue_light, myblue_dark]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(t, gij_PEG, 's', markersize=msize, markeredgecolor=edge, markeredgewidth=0.2, color=myred, label=r'PEG')
    ax1.loglog(t, gij_H2O, 'o', markersize=msize, markeredgecolor=edge, markeredgewidth=0.2, color=myblue, label=r'H2O')
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$t$ (ps)", ylabel=r'$G^{(0)} (t) / G^{(0)} (0)$',)
                       #xlim=(1, 1e3), ylim=(1e6, 1e11))
    save_figure(plt, fig, mode, name='tutorials/polymer-in-water/Gij', save=True)